## Using Kaggles Train Play Tennis Dataset to perform an ID3 algrorithim

Adam Ruane - Information based Learning 

ID3 (Iterative Dichotomiser 3) is a popular algorithm used for constructing decision trees in machine learning and data mining. It was developed by Ross Quinlan in the 1980s and is one of the foundational algorithms for decision tree learning. Decision trees are used for classification and regression tasks and are particularly useful for tasks involving structured and categorical data. ID3 is primarily used for classifica-tion tasks. I will be lookimng at Data Preperation, Entropy (the impurity of the dataset) and Information Gain (How much the entrophy decraeses after attribbute is split)

Link of the dataset: https://www.kaggle.com/tareqjoy/trainplaytennis

Imports 

In [30]:
import pandas as pd
import numpy as np

Read the Dataset 

In [31]:
df = pd.read_csv("PlayTennis_train.csv")

Inspection

In [32]:
df.head()

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [33]:
# let's inspect:
df.shape 

(14, 5)

Attributes: Attributes are the variables or features that describe the data. In the context of the classic Weather dataset used in ID3, typical attributes might include things like Outlook (sunny, overcast, rainy), Temperature (hot, mild, cool), Humidity (high, normal), and Wind (strong, weak).

Attribute Values: These are the possible values each attribute can take. For example, for the Outlook attribute, the possible values are "sunny," "overcast," and "rainy." For Temperature, it could be "hot," "mild," and "cool."

Training Cases: Training cases are the individual examples or instances in the dataset. Each training case represents a combination of attribute values along with a corresponding class.

Identifiers: Identifiers are unique values or keys assigned to each training case to distinguish them from one another. In the example dataset, there might not be explicit identifiers, or they could be represented as row numbers or indices. (usually not relevant for training)

Classes: Classes are the target labels or outcomes that the machine learning model aims to predict. In this dataset, the "Play" attribute represents the class labels, which can be either "yes" or "no." The goal is to predict whether to play or not based on the given attributes.



Features: Outlook, Temperature, Humidity, Wind
Label: Play Tennis (The output feature that we want to predict)
Class: Yes, No (Unique values of the label)


Total row = 14
Row with "Yes" class = 9
Row with "No" class = 5
Complete entropy of dataset is -
H(S) = - p(Yes) * log2(p(Yes)) - p(No) * log2(p(No))
     = - (9/14) * log2(9/14) - (5/14) * log2(5/14)
     = - (-0.41) - (-0.53)
     = 0.94

## Entrophy 

In [20]:

def calc_total_entropy(train_data, label, class_list):
    total_row = train_data.shape[0] #the total size of the dataset
    total_entr = 0
    
    for c in class_list: #for each class in the label
        total_class_count = train_data[train_data[label] == c].shape[0] #number of the class
        total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row) #entropy of the class
        total_entr += total_class_entr #adding the class entropy to the total entropy of the dataset
    
    return total_entr

Method description:
Calculates entropy of a specific feature = value.

In [21]:
def calc_entropy(feature_value_data, label, class_list):
    class_count = feature_value_data.shape[0]
    entropy = 0
    
    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0] #row count of class c 
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count #probability of the class
            entropy_class = - probability_class * np.log2(probability_class)  #entropy
        entropy += entropy_class
    return entropy

Information Gain 

In [22]:

def calc_info_gain(feature_name, train_data, label, class_list):
    feature_value_list = train_data[feature_name].unique() #unqiue values of the feature
    total_row = train_data.shape[0]
    feature_info = 0.0
    
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value] #filtering rows with that feature_value
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list) #calculcating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy #calculating information of the feature value
        
    return calc_total_entropy(train_data, label, class_list) - feature_info #calculating information gain by subtracting

Findig the most informative feature 

In [23]:
def find_most_informative_feature(train_data, label, class_list):
    feature_list = train_data.columns.drop(label) #finding the feature names in the dataset
                                            #N.B. label is not a feature, so dropping it
    max_info_gain = -1
    max_info_feature = None
    
    for feature in feature_list:  #for each feature in the dataset
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain: #selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature
            
    return max_info_feature

Add a node to the tree 
As we have found the feature name with the highest information gain, we have to generate a node in the tree and its value as a branch.

In [35]:
def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False).to_dict()  # Convert to dictionary
    tree = {}  # Sub tree or node
    
    for feature_value, count in feature_value_count_dict.items():  # Use .items() instead of .iteritems()
        feature_value_data = train_data[train_data[feature_name] == feature_value]  # Dataset with only feature_name = feature_value
        
        assigned_to_node = False  # Flag for tracking if feature_value is pure class or not
        for c in class_list:  # For each class
            class_count = feature_value_data[feature_value_data[label] == c].shape[0]  # Count of class c

            if class_count == count:  # If count of (feature_value = count) of class is equal to total count (pure class)
                tree[feature_value] = c  # Adding node to the tree
                train_data = train_data[train_data[feature_name] != feature_value]  # Removing rows with feature_value
                assigned_to_node = True
        if not assigned_to_node:  # If not pure class
            tree[feature_value] = "?"  # As feature_value is not a pure class, it should be expanded further, so mark the branch with '?'
            
    return tree, train_data


Performing ID3 Algorithm and generating Tree
Now, we should ensemble the methods which should recursively do. So, the overall step is:

Finding the most informative feature
Making a tree node with a feature name and feature values as branches
- If pure class, adding leaf node (= Class) to the tree node
- If impure class, adding an expandable node (= ‘?’) to the tree node
Shrinking/Updating the dataset according to the pure class
Adding the node with branches into a tree
Expand the branch of the next impure class (= ‘?’) with an updated dataset
The recursion endpoint:

The dataset becomes empty after updating
There is no expandable branch (= all pure class)

In [36]:

def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0: #if dataset becomes enpty after updating
        max_info_feature = find_most_informative_feature(train_data, label, class_list) #most informative feature
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list) #getting tree node and updated dataset
        next_root = None
        
        if prev_feature_value != None: #add to intermediate node of the tree
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else: #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()): #iterating the tree node
            if branch == "?": #if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == node] #using the updated dataset
                make_tree(next_root, node, feature_value_data, label, class_list) #recursive call with updated dataset

Now to put the ID3 algorithim together 

In [37]:
def id3(train_data_m, label):
    train_data = train_data_m.copy() #getting a copy of the dataset
    tree = {} #tree which will be updated
    class_list = train_data[label].unique() #getting unqiue classes of the label
    make_tree(tree, None, train_data, label, class_list) #start calling recursion
    return tree

Run the algorithim on our data 

In [38]:
tree = id3(df, 'Play Tennis')

In [39]:
tree

{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}},
  'Overcast': 'Yes',
  'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}

Testing 

In [40]:
def predict(tree, instance):
    if not isinstance(tree, dict): #if it is leaf node
        return tree #return the value
    else:
        root_node = next(iter(tree)) #getting first key/feature name of the dictionary
        feature_value = instance[root_node] #value of the feature
        if feature_value in tree[root_node]: #checking the feature value in current tree node
            return predict(tree[root_node][feature_value], instance) #goto next feature
        else:
            return None

Evaluation 

In [41]:

def evaluate(tree, test_data_m, label):
    correct_preditct = 0
    wrong_preditct = 0
    for index, row in test_data_m.iterrows(): #for each row in the dataset
        result = predict(tree, test_data_m.iloc[index]) #predict the row
        if result == test_data_m[label].iloc[index]: #predicted value and expected value is same or not
            correct_preditct += 1 #increase correct count
        else:
            wrong_preditct += 1 #increase incorrect count
    accuracy = correct_preditct / (correct_preditct + wrong_preditct) #calculating accuracy
    return accuracy

In [42]:
test_data_m = pd.read_csv("PlayTennis_test.csv") #importing test dataset into dataframe

accuracy = evaluate(tree, test_data_m, 'Play Tennis') #evaluating the test dataset

In [45]:
print(f"Accuracy is {accuracy * 100} %" )

Accuracy is 100.0 %
